In [1]:
# import tensorflow as tf

# x = tf.Variable(0, name='counter')
# assign_op = tf.assign(x, 0)
# p = tf.placeholder(tf.int32, ())
# with tf.control_dependencies([assign_op]):
#     x = tf.assign_add(x, p, name='increment')
#     x = tf.assign_add(x, p, name='increment_2')

# with tf.Session() as sess:
#     # writer = tf.summary.FileWriter('logs', sess.graph)
#     sess.run(tf.global_variables_initializer())
#     for _ in range(5):
#         print(sess.run(x, {p: 1}))

## test batchsize to be None

In [2]:
import tensorflow as tf
import numpy as np
from utils import batch_scatter_add

batch_size = 2
num_ions = 2
hidden_size = 3
M = 5

hr_np = np.arange(batch_size * 1 * num_ions * hidden_size).reshape(batch_size,1,num_ions,hidden_size).astype(np.float32)
location_np = np.array([[0,4], [2, 4]]).astype(np.int32).reshape(2,1,2)
location_index = tf.placeholder(tf.int32, (None, 1, num_ions), name='location')
#ion_hidden_represent = tf.placeholder(tf.float32, (batch_size, 1, num_ions, hidden_size), name='hidden_representation')
ion_hidden_represent = tf.Variable(hr_np, name='hidden_represent', trainable=True)
resized_ion_hidden = tf.reshape(ion_hidden_represent, (-1, num_ions, hidden_size))
resized_location_index = tf.reshape(location_index, (-1, num_ions))

output = tf.zeros((batch_size, M, hidden_size), dtype=tf.float32)

gathered_output = tf.batch_gather(params=output, indices=resized_location_index) #(batch, resize_location_index, hidden_size)
update = resized_ion_hidden
output = batch_scatter_add(ref=output, indices=resized_location_index, updates=update, name='batch_scatter_add')
final_output = tf.reduce_mean(output)
gradient = tf.gradients(final_output, output)
gradient_to_hidden = tf.gradients(final_output, ion_hidden_represent)[0]


In [3]:
with tf.Session() as sess:
    init_op = tf.initializers.variables(tf.global_variables())
    sess.run(init_op)
    #print(sess.run(gathered_output, feed_dict={location_index:location_np}))
    print(sess.run(final_output, feed_dict={location_index:location_np}))
    print(sess.run(final_output, feed_dict={location_index:location_np}))
    print(sess.run(output, feed_dict={location_index:location_np}))
    print(sess.run(gradient_to_hidden, feed_dict={location_index:location_np}))

2.2
2.2
[[[ 0.  1.  2.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 3.  4.  5.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 6.  7.  8.]
  [ 0.  0.  0.]
  [ 9. 10. 11.]]]
[[[[0.03333334 0.03333334 0.03333334]
   [0.03333334 0.03333334 0.03333334]]]


 [[[0.03333334 0.03333334 0.03333334]
   [0.03333334 0.03333334 0.03333334]]]]


In [7]:
np.allclose(np.zeros(2), np.zeros(2))

True